### Import full Exiobase for 2019

In [ ]:
import pymrio
import os
import iode as io
import numpy as np
import pandas as pd
import re 

# Define the working path
wk_path = "to define"

# Load previously extracted and saved 
save_folder_full = wk_path + "Data_Raw/EXIOBASE/pymrio/"
exio19 = pymrio.load_all(path=save_folder_full)

# Initial list and values
BASEYEAR = "2019Y1"
LOADYEAR = "2019Y1"
SMPSTRY = "2015Y1"
SMPENDY = "2050Y1"
SMP = SMPSTRY + ":" + SMPENDY
print(SMP)
io.variables.sample = f'{SMP}'

2015Y1:2050Y1


### Aggregation of initial database

In [ ]:
# Caution does not work when running the code several times, mrio base must be reloaded
# Modify codes lists

## Rename final demands indexes
lst_ini_FD = exio19.get_Y_categories()
# Final consumption expenditure by households: CONSHV
# Final consumption expenditure by non-profit organisations serving households (NPISH): CONSNPV
# Final consumption expenditure by government: CONSGV
# Gross fixed capital formation: DINVV
# Changes in inventories: DSTOCKV
# Changes in valuables: DVALUEV
# Exports: Total (fob): EXPTOTV
lst_upd_FD = ["CONSHV","CONSNPV","CONSGV","DINVV","DSTOCKV","DVALUEV","EXPTOTV"]
dict_FD = dict(zip(lst_ini_FD, lst_upd_FD))
exio19.rename_Y_categories(dict_FD)

## Rename sectors only with numbers (3-digit)
# Do a list of numbers from 1 to 164 with zero(s) before
lst_upd_Sect = [str(i).zfill(3) for i in range(1, 164)]
lst_ini_Sect = exio19.get_sectors()
dict_Sect = dict(zip(lst_ini_Sect, lst_upd_Sect))
exio19.rename_sectors(dict_Sect)

## Mapping of the Sectors
map_sect = pd.read_excel(os.path.join(save_folder_full,"Mapping_Sectors.xlsx"), dtype=str) # Excel file containing sectors' mapping for aggregation
map_ini_sect = map_sect["Sect_cd_ini"]          # Initial sectors
map_upd_sect = map_sect["Sect_cd_final"]        # Aggregated sectors
dict_map_sect = dict(zip(map_ini_sect, map_upd_sect)) 
exio19.rename_sectors(dict_map_sect)            # Replace names
exio19.aggregate_duplicates()                   # Groupby

## Mapping of the regions
map_reg = pd.read_excel(os.path.join(save_folder_full,"Mapping_Regions.xlsx"), dtype=str) # Excel file containing regions' mapping for aggregation
map_ini_reg = map_reg["Reg_cd_ini"]             # Initial regions
map_upd_reg = map_reg["Reg_cd_final"]           # Aggregated regions
dict_map_reg = dict(zip(map_ini_reg, map_upd_reg))
exio19.rename_regions(dict_map_reg)             # Replace names
exio19.aggregate_duplicates()                   # Groupby



#### Loading calculated GFCF matrix

In [ ]:
# Define working paths
path_mat_inv = wk_path + "/Data_Raw/GFCF/"
# Load GFCF Intensity to allocate total GFCF to sector (GFCF intensity have been calculated previously, used here as input) 
data_mat_inv = os.path.join(path_mat_inv, "Inv_mat_balanced.csv")
mat_gfcf = pd.read_csv(data_mat_inv, sep=";", dtype={1: str, 0: str}, header=[0, 1], index_col=[0, 1])
# to transform index value 1,2,3 etc; into 01,02,03 etc.
mat_gfcf.index = mat_gfcf.index.set_levels(mat_gfcf.index.levels[1].map(lambda x: f"{int(x):02}"), level=1)
# Calculation of DINVV
mat_gfcf_dinv = mat_gfcf.sum(axis=0)
# Calculation of ADDDINVV
mat_gfcf_addinv = mat_gfcf.T.groupby("col_reg").sum()
#print(mat_gfcf_addinv.head())
print(mat_gfcf_addinv.head())
print(mat_gfcf_dinv.head())

### Load scalars to calculate KSTOCK (based on average sectoral capital stock intensity, over production)

In [ ]:
## Load scalars of intensity
# Define working paths
path_kstock = wk_path + "Data_Raw/KSTOCK/"
# Load GFCF intensity to allocate total GFCF to sector 
scl_kstock = os.path.join(path_kstock, "Scl_int_kstock.csv")
int_kstock = pd.read_csv(scl_kstock , sep=";", header=[0], dtype={"Sec_cd": str})


### Main variables calculation from EXIOBASE

In [ ]:
dict_var = dict() # Dictionnay that will store all variables
val = {f"{year}Y1": np.nan for year in range(int(SMPSTRY[:-2]), int(SMPENDY[:-2])+1)} # dictionnay to store variables' value according to pre-define sample

####################################################
# Final demands
##### DINVV
# Use GFCF calculated previously
# Store values into the dictionnary
for rd in mat_gfcf_dinv.index.get_level_values("col_reg").unique():
    for sd in mat_gfcf_dinv.index.get_level_values("col_sec").unique():
        val[f"{BASEYEAR}"] = mat_gfcf_dinv.loc[(rd,sd),]
        dict_var[f"DINVV_{rd}_{sd}"] = {"val": val.copy(), "cmt": f"Gross fixed capital formation, nominal", "unit": "current million euros", "idt": "", "eqs": ""}

# To verify that sum of all ADDINVV equals DINVV
print(exio19.Y.loc[:,(exio19.Y.columns.get_level_values("category") == "DINVV")].sum().T.sum(), mat_gfcf_dinv.sum())

##### DSTOCKV and DVALUEV
# Caution summation of final demands : DSTOCKV and DVALUEV in one DSTOCKV
DSTOCKV = exio19.Y.loc[:,(exio19.Y.columns.get_level_values("category") == "DSTOCKV")].groupby(level=1).sum()
DVALUEV = exio19.Y.loc[:,(exio19.Y.columns.get_level_values("category") == "DVALUEV")].groupby(level=1).sum()
# Concatenation of both dataframe
DSTOCKV_sum = pd.concat([DSTOCKV, DVALUEV], axis=1)
# Sum for each region in columns both final demands 
DSTOCKV = DSTOCKV_sum.T.groupby("region", sort=False).sum().T
# Add an colunm index "category" with the value "DSTOCK"
n_col = pd.MultiIndex.from_product([DSTOCKV.columns, ["DSTOCKV"]], names=['region', 'category'])
DSTOCKV.columns = n_col

###### OTHER FINAL DEMANDS
# Extract values for each final demand to create variable with dimension region and sector
for rd in exio19.Y.columns.get_level_values("region").unique():
    for ss in exio19.Y.index.get_level_values("sector").unique():
        # caution: it excludes the three last elements of the lst: DINVV (already calculated) DSTOCKV & DVALUEV (sumed with DSTOCKV in df DSTOCKV) & EXPTOTV (always equals zero)
        for var, cmt in zip(lst_upd_FD[:-4], lst_ini_FD[:-4]):
            df_temp = exio19.Y.loc[:,exio19.Y.columns.get_level_values("category") == f"{var}"].groupby(level=1).sum()
            val[f"{BASEYEAR}"] = df_temp.loc[ss, (rd,var)]            
            dict_var[f"{var}_{rd}_{ss}"] = {"val": val.copy(), "cmt": f"{cmt}, nominal", "unit": "current million euros", "idt": "", "eqs": ""}
        val[f"{BASEYEAR}"]  = DSTOCKV.loc[ss, (rd, f"{lst_upd_FD[4]}")] 
        dict_var[f"{lst_upd_FD[4]}_{rd}_{ss}"] = {"val": val.copy() , "cmt": f"{lst_ini_FD[4]}, nominal", "unit": "current million euros", "idt": "", "eqs": ""}
        del df_temp


##################################################
# Total Intermediate consumption - DINTCONV
# Sum to calculate variables
DINTCONSV = exio19.Z.sum()

### CAUTION VERIFY AXIS SUMMATION
# Energy intermediate consumption - DENRTOTV ()
# Caution no bioenergy accounted from crops sector
lst_enr_sect = ["05","06","07","13","14","35","36","37","38","39"]
# Sum demands to energy supply sectors for each region 
DENRTOTV = exio19.Z.loc[(exio19.Z.index.get_level_values('sector').isin(lst_enr_sect))].groupby("sector").sum().sum()

# Calculation of the intermediate consumption demand excluding energy consumption
DMATV = DINTCONSV-DENRTOTV

# Create the variables for each region and each sector                
for r in DINTCONSV.index.get_level_values("region").unique():
    for s in DINTCONSV.index.get_level_values("sector").unique():
        val[f"{BASEYEAR}"] = DINTCONSV.loc[(r,s)]
        dict_var[f"DINTCONSV_{r}_{s}"] = {"val": val.copy(), "cmt": "Total intermediate consumption, nominal", "unit": "current million euros", "idt": "", "eqs": ""}
        val[f"{BASEYEAR}"]  = DENRTOTV.loc[(r,s)]
        dict_var[f"DENRTOTV_{r}_{s}"] = {"val": val.copy(), "cmt": "Total energy consumption, nominal", "unit": "current million euros", "idt": "", "eqs": ""}
        val[f"{BASEYEAR}"]  = DMATV.loc[(r,s)]
        dict_var[f"DMATV_{r}_{s}"] = {"val": val.copy(), "cmt": "Total intermediate consumption except energy, nominal", "unit": "current million euros", "idt": "", "eqs": ""}


#######################################################
# Production - PRODV
PRODV = exio19.x
for r in PRODV.index.get_level_values("region").unique():
    for s in PRODV.index.get_level_values("sector").unique():
            val[f"{BASEYEAR}"]  = PRODV.loc[(r,s), "indout"]
            dict_var[f"PRODV_{r}_{s}"] = {"val": val.copy(), "cmt": "Production, nominal", "unit": "current million euros", "idt": "", "eqs": ""}


# Capital Stock - KSTOCK (using scalar on intenisity previoulsy loaded)
for r in PRODV.index.get_level_values("region").unique():
    for s in PRODV.index.get_level_values("sector").unique():
        val[f"{BASEYEAR}"]  = PRODV.loc[(r,s), "indout"]*int_kstock.loc[(int_kstock["Reg_cd"] == r) & (int_kstock["Sec_cd"] == s), "int_kstock"].sum()
        dict_var[f"KSTOCK_{r}_{s}"] = {"val": val.copy(), "cmt": "Capital stock", "unit": "current million euros", "idt": "", "eqs": ""}


# Value added - VAMPV
VAMPV = exio19.impacts.F.loc[exio19.impacts.F.index.get_level_values("impact") == "Value Added"]
for r in VAMPV.columns.get_level_values("region").unique():
    for s in VAMPV.columns.get_level_values("sector").unique():
            val[f"{BASEYEAR}"] = VAMPV.loc["Value Added", (r,s)]
            dict_var[f"VAMPV_{r}_{s}"] = {"val": val.copy(), "cmt": "Value Added at market price, nominal", "unit": "current million euros", "idt": "", "eqs": ""}


#########################################################
# Variables from Satellite account
# Maps variables from Satelite accounts with NeW variables
lst_vars_stress = ["Taxes less subsidies on products purchased: Total",
"Other net taxes on production",
"Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled",
"Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled",
"Compensation of employees; wages, salaries, & employers' social contributions: High-skilled",
"Operating surplus: Consumption of fixed capital",
"Operating surplus: Rents on land",
"Operating surplus: Royalties on resources",
"Operating surplus: Remaining net operating surplus",
"Employment: Low-skilled male",
"Employment: Low-skilled female",
"Employment: Medium-skilled male",
"Employment: Medium-skilled female",
"Employment: High-skilled male",
"Employment: High-skilled female",
"Employment hours: Low-skilled male",
"Employment hours: Low-skilled female",
"Employment hours: Medium-skilled male",
"Employment hours: Medium-skilled female",
"Employment hours: High-skilled male",
"Employment hours: High-skilled female",
"Employment: Vulnerable employment",
"Employment hours: Vulnerable employment"]
lst_vars_NeW_stress = ["OTHIMPSUBPRD","TAXPRUDV","CMPEMPALS","CMPEMPAMS","CMPEMPAHS",
"GOPCONSKV","GOPRENTLV","GOPRENTRESV","GOPOTHV",
"DEMPTOTLS_F","DEMPTOTLS_M","DEMPTOTMS_F","DEMPTOTMS_M","DEMPTOTHS_F","DEMPTOTHS_M",
"DEMPHOURLS_F","DEMPHOURLS_M","DEMPHOURMS_F","DEMPHOURMS_M","DEMPHOURHS_F","DEMPHOURHS_M",
"DEMPTOTRISK","DEMPHOURRISK"]

# Add the variable into the dictionnay of data
for r in exio19.satellite.F.columns.get_level_values("region").unique():
    for s in exio19.satellite.F.columns.get_level_values("sector").unique():
            for vars, stress in zip(lst_vars_NeW_stress, lst_vars_stress):
                df_temp = exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == f"{stress}"]
                val[f"{BASEYEAR}"]  = df_temp.loc[f"{stress}", (r,s)]
                if "Employment hours:" in stress: 
                    dict_var[f"{vars}_{r}_{s}"] = {"val": val.copy(), "cmt": stress, "unit": "million hours", "idt": "", "eqs": ""}
                else:
                    if "Employment:" in stress:
                        dict_var[f"{vars}_{r}_{s}"] = {"val": val.copy(), "cmt": stress, "unit": "thousand persons", "idt": "", "eqs": ""}
                    else:
                         dict_var[f"{vars}_{r}_{s}"] = {"val": val.copy(), "cmt": stress, "unit": "million current euros", "idt": "", "eqs": ""}

                    

# Completion of the data with agregates, etc...
CMPEMPA = pd.concat([exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Compensation of employees; wages, salaries, & employers' social contributions: High-skilled"]]).sum()
DEMPTOTLS = pd.concat([exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: Low-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: Low-skilled female"]]).sum()
DEMPTOTMS = pd.concat([exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: Medium-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: Medium-skilled female"]]).sum()
DEMPTOTHS = pd.concat([exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: High-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: High-skilled female"]]).sum()
DEMPTOT = pd.concat([exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: Low-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: Low-skilled female"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: Medium-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: Medium-skilled female"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: High-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment: High-skilled female"]]).sum()
DEMPHOURLS = pd.concat([exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: Low-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: Low-skilled female"]]).sum()
DEMPHOURMS = pd.concat([exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: Medium-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: Medium-skilled female"]]).sum()
DEMPHOURHS = pd.concat([exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: High-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: High-skilled female"]]).sum()
DEMPHOUR = pd.concat([exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: Low-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: Low-skilled female"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: Medium-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: Medium-skilled female"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: High-skilled male"],
                    exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor") == "Employment hours: High-skilled female"]]).sum()

lst_vars_stress_agg =["Compensation of employees: wages, salaries, & employers' social contributions",             
"Employment: Low-skilled","Employment: Medium-skilled","Employment: High-skilled", "Employment",
"Employment hours: Low-skilled","Employment hours: Medium-skilled","Employment hours: High-skilled","Employment hours"]
lst_vars_NeW_stress_agg = ["CMPEMPA","DEMPTOTLS","DEMPTOTMS","DEMPTOTHS","DEMPTOT","DEMPHOURLS","DEMPHOURMS","DEMPHOURHS","DEMPHOUR"]

for r in exio19.satellite.F.columns.get_level_values("region").unique():
    for s in exio19.satellite.F.columns.get_level_values("sector").unique():
            for vars, stress in zip(lst_vars_NeW_stress_agg, lst_vars_stress_agg):
                val[f"{BASEYEAR}"]  = globals()[f"{vars}"].loc[(r,s)]
                if "Employment hours" in stress: 
                    dict_var[f"{vars}_{r}_{s}"] = {"val": val.copy(), "cmt": stress, "unit": "million hours", "idt": "", "eqs": ""}
                else:
                    if "Employment" in stress:
                        dict_var[f"{vars}_{r}_{s}"] = {"val": val.copy(), "cmt": stress, "unit": "thousand persons", "idt": "", "eqs": ""}
                    else:
                         dict_var[f"{vars}_{r}_{s}"] = {"val": val.copy(), "cmt": stress, "unit": "million current euros", "idt": "", "eqs": ""}



### Trade calculation : Imports & exports

In [ ]:
### Bilateral trade
## Calculation of the Intermediate consumption imported 
# Caution: use .copy() to avoid overwriting the initial dataframe
trade_ci = exio19.Z.copy()
# Select only energy demands
trade_ci_enr = exio19.Z.loc[exio19.Z.index.get_level_values('sector').isin(lst_enr_sect)].copy()
# Reindex the enr dataframe with ci dtataframe index and add 0 when none existing values
trade_ci_enr = trade_ci_enr.reindex(trade_ci.index, fill_value=0)
# Calculation of ci without ener
trade_ci_mat = trade_ci-trade_ci_enr

# Replace values by 0 for domestic intermediate consumption
for region in trade_ci.index.get_level_values('region').unique():
    trade_ci.loc[region, region] = 0
    trade_ci_enr.loc[region, region] = 0
    trade_ci_mat.loc[region, region] = 0
# Sum the supplier sectors by supplier region
trade_ci_bilat = trade_ci.T.groupby("region", sort=False).sum().T
trade_ci_enr_bilat = trade_ci_enr.T.groupby("region", sort=False).sum().T
trade_ci_mat_bilat = trade_ci_mat.T.groupby("region", sort=False).sum().T

# Calculation of the Final demands imported
# Caution: use .copy() to avoid overwriting the initial dataframe
# Create a new dataframe summing DSTOCK and DVALUE in DSTOCK and delete EXPTOTV
df_fd = exio19.Y.copy()
for reg in exio19.Y.columns.get_level_values("region").unique():
    df_fd.loc[:,(reg,"DSTOCKV")] =  exio19.Y.loc[:,(reg,"DSTOCKV")]+exio19.Y.loc[:,(reg,"DVALUEV")] 
df_fd = df_fd.loc[:, (df_fd.columns.get_level_values("category") != "DVALUEV") & (df_fd.columns.get_level_values("category") != "EXPTOTV")]

trade_fd = df_fd.copy()
trade_fd_enr = df_fd.loc[df_fd.index.get_level_values('sector').isin(lst_enr_sect)].copy()
trade_fd_enr = trade_fd_enr.reindex(trade_fd.index, fill_value=0)
trade_fd_mat = trade_fd-trade_fd_enr

# Replace values by 0 for domestic final demands
for region in trade_fd.index.get_level_values('region').unique():
    trade_fd.loc[region, region] = 0
    trade_fd_enr.loc[region, region] = 0
    trade_fd_mat.loc[region, region] = 0
# Sum all final demands    
trade_fd_bilat = trade_fd.T.groupby("region", sort=False).sum().T
trade_fd_enr_bilat = trade_fd_enr.T.groupby("region", sort=False).sum().T
trade_fd_mat_bilat = trade_fd_mat.T.groupby("region", sort=False).sum().T

# Addition of imported intermediate consumption and imported final demands (rows: exports, columns: imports)
trade_bilat = trade_ci_bilat+trade_fd_bilat
trade_enr_bilat = trade_ci_enr_bilat+trade_fd_enr_bilat
trade_mat_bilat = trade_ci_mat_bilat+trade_fd_mat_bilat

# Total exports by region and sector
exp_tot = trade_bilat.sum(axis=1)
exp_tot_enr = trade_enr_bilat.sum(axis=1)
exp_tot_mat = trade_mat_bilat.sum(axis=1)
# Total imports (with reallocation of columns index into row: stack(), change row indexes order: swaplevel())
imp_tot = trade_bilat.groupby("sector", sort=False).sum().stack().swaplevel()
imp_tot_enr = trade_enr_bilat.groupby("sector", sort=False).sum().stack().swaplevel()
imp_tot_mat = trade_mat_bilat.groupby("sector", sort=False).sum().stack().swaplevel()


# Add bilateral imports in the dictionnary contening variables
for rd in trade_bilat.index.get_level_values("region").unique():
    for sd in trade_bilat.index.get_level_values("sector").unique():
            for rs in trade_bilat.index.get_level_values("region").unique():
                val[f"{BASEYEAR}"]  = trade_bilat.loc[(rd,sd),rs]
                dict_var[f"EXPTOTV_{rd}_{sd}_{rs}"]  = {"val": val, "cmt": f"Exports from {rd} to {rs}, nominal", "unit": "million current euros", "idt": "", "eqs": ""}
            val[f"{BASEYEAR}"] = trade_bilat.loc[(rd,sd)].sum()
            dict_var[f"EXPTOTV_{rd}_{sd}"]  = {"val": val.copy(), "cmt": f"Exports from {rd}, nominal", "unit": "million current euros", "idt": "", "eqs": ""}
            


### Calculation of the framework of addressed demands
###### (All domestic demands, by type of demand, for industry ss is adressed to domectic sector sd that will thereafter choice to produce or import)

In [ ]:
#####################################
### Intermediate consumption
# Total domestic demand
# Demand of region (rd) and sector (sd) to supplying industry (ss) whatever the region of origin (sum of rs)
addci_tot = exio19.Z.groupby("sector", sort=False).sum()
addci_enr_tot = addci_tot.loc[addci_tot.index.get_level_values('sector').isin(lst_enr_sect)]
addci_enr_tot = addci_enr_tot.reindex(addci_tot.index, fill_value=0) 
addci_mat_tot = addci_tot-addci_enr_tot

# Calculate the total intermediate demand addressed to industry "ss" by all industries (sum of sd) from domestic region (rd)
addci_dom = []
addci_enr_dom = []
addci_mat_dom = []
for rd in addci_tot.columns.get_level_values("region").unique():
    for ss in addci_tot.index.get_level_values("sector").unique():
        addci_dom.append(addci_tot.loc[ss,rd].sum())
        addci_enr_dom.append(addci_enr_tot.loc[ss,rd].sum())
        addci_mat_dom.append(addci_mat_tot.loc[ss,rd].sum())
addci_dom = pd.Series(addci_dom, index=addci_tot.columns)
addci_enr_dom = pd.Series(addci_enr_dom, index=addci_enr_tot.columns)
addci_mat_dom = pd.Series(addci_mat_dom, index=addci_mat_tot.columns)

# Imported domestic
# Imported demand of region (rd) and sector (rd) to supplying industry (ss) whatever the region of origin expect the domestic region (sum of rs != rd)
addciimp_tot = trade_ci.groupby("sector", sort=False).sum()
addciimp_enr_tot = trade_ci_enr.groupby("sector", sort=False).sum()
addciimp_mat_tot = addciimp_tot-addciimp_enr_tot

# Calculate the total imported intermediate demand addressed to industry "ss" by all industries (sum of sd) from domestic region (rd)
addciimp_dom = []
addciimp_enr_dom = []
addciimp_mat_dom = []
for rd in addciimp_tot.columns.get_level_values("region").unique():
    for ss in addciimp_tot.index.get_level_values("sector").unique():
        addciimp_dom.append(addciimp_tot.loc[ss,rd].sum())
        addciimp_enr_dom.append(addciimp_enr_tot.loc[ss,rd].sum())
        addciimp_mat_dom.append(addciimp_mat_tot.loc[ss,rd].sum())
addciimp_dom = pd.Series(addciimp_dom, index=trade_ci.columns)
addciimp_enr_dom = pd.Series(addciimp_enr_dom, index=trade_ci_enr.columns)
addciimp_mat_dom = pd.Series(addciimp_mat_dom, index=trade_ci_mat.columns)

# Add the variables into the model dataset (total intermediate consumption is not included)
for r in addci_dom.index.get_level_values("region").unique():
     for s in addci_dom.index.get_level_values("sector").unique():
        val[f"{BASEYEAR}"]  = addci_enr_dom.loc[r,s]
        dict_var[f"ADDDENRTOTV_{r}_{s}"]  = {"val": val.copy(), "cmt": "Total addressed energy demand, nominal", "unit": "million current euros", "idt": "", "eqs": ""}
        val[f"{BASEYEAR}"]  = addci_mat_dom.loc[r,s]
        dict_var[f"ADDDMATV_{r}_{s}"]  = {"val": val.copy(), "cmt": "Total addressed intermediate consumption (except energy) demand, nominal", "unit": "million current euros", "idt": "", "eqs": ""}
        val[f"{BASEYEAR}"]  = addciimp_enr_dom.loc[r,s]
        dict_var[f"IMPDENRTOTV_{r}_{s}"]  = {"val": val.copy(), "cmt": "Imported energy demand, nominal", "unit": "million current euros", "idt": "", "eqs": ""}
        val[f"{BASEYEAR}"]  = addciimp_mat_dom.loc[r,s]
        dict_var[f"IMPDMATV_{r}_{s}"]  = {"val": val.copy(), "cmt": "Imported intermediate consumption (except energy) demand, nominal", "unit": "million current euros", "idt": "", "eqs": ""}
        

####################################
### Final demands
# Total domestic demand
# Final demand of region (rd) and sector (sd) to supplying industry (ss) whatever the region of origin (sum of rs)
addfd_temp = df_fd.groupby("sector", sort=False).sum()
# For each final demand, final demand addressed to industry "ss" by domestic region (rd) (permutation opf the previous dataframe)
addfd_dom = pd.DataFrame(index=df_fd.index, columns=addfd_temp.columns.get_level_values("category").unique())
for fd in addfd_temp.columns.get_level_values("category").unique():
    for r in addfd_temp.columns.get_level_values("region").unique():
        for s in addfd_temp.index.get_level_values("sector").unique():
            addfd_dom.loc[(r,s),fd] = addfd_temp.loc[s,(r,fd)]



# Imported final demands
# Final demand of region (rd) and sector (sd) to supplying industry (ss) whatever the region of origin expect the domestic region (sum of rs != rd)
addfdimp_tot = trade_fd.groupby("sector", sort=False).sum()
# For each final demand, imported final demand addressed to industry "ss" by domestic region (rd) (permutation of the previous dataframe)
addfdimp_dom = pd.DataFrame(index= trade_fd.index, columns=addfdimp_tot.columns.get_level_values("category").unique())
for fd in addfdimp_tot.columns.get_level_values("category").unique():
    for rd in addfdimp_tot.columns.get_level_values("region").unique():
        for ss in addfdimp_tot.index.get_level_values("sector").unique():
            addfdimp_dom.loc[(rd,ss),fd] = addfdimp_tot.loc[ss,(rd,fd)]

# Add the variables into the model dataset
for fd, cmt in zip(addfd_dom.columns.get_level_values("category").unique(),lst_ini_FD[:-2]):
    for r in addfd_dom.index.get_level_values("region").unique():
        for s in addfd_dom.index.get_level_values("sector").unique():
            val[f"{BASEYEAR}"] = addfd_dom.loc[(r,s), fd]
            dict_var[f"ADD{fd}_{r}_{s}"]  = {"val": val.copy(), "cmt":f"Addressed {cmt.lower()}, nominal", "unit": "million current euros", "idt": "", "eqs": ""}
            val[f"{BASEYEAR}"] = addfdimp_dom.loc[(r,s),fd]
            dict_var[f"IMP{fd}_{r}_{s}"] = {"val": val.copy(), "cmt": f"Imported {cmt.lower()}, nominal", "unit": "million current euros", "idt": "", "eqs": ""}



#### Store variables and comments into iode objects

In [ ]:
for key in dict_var.keys():
    io.variables[f"{key}"] = np.nan
    io.variables[f"{key}", f"{LOADYEAR}"] = dict_var[f"{key}"]["val"][f"{LOADYEAR}"]
    io.comments[f"{key}"] = dict_var[f"{key}"]["cmt"] + "; Unit:" + dict_var[f"{key}"]["unit"]  
   

### Calculation of HHs accounts

In [ ]:
dict_var_sa = dict()

# Loading pre-calculated scalars (from OECD "" and processed)
path_scl_hhinc = wk_path + "Data_Raw/Non-Fin_Accounts/"
csv_scl_hhinc = os.path.join(path_scl_hhinc, "HH_scl_inc_2019.csv")
scl_hhinc = pd.read_csv(csv_scl_hhinc, sep=";", dtype={0: str}, header=[0], index_col=[0])

# Calculation of variables proportional to CMPEMPA
# Sum all CMPEMPA by country
CMPEMPA_r = CMPEMPA.groupby("region").sum()
# Variables and comments
dict_var_cmph = {"CMPEMREH": "Compensation of employess (D1), received by households", "CMPEMPAH": "Compensation of employess (D1), paid by households", 
                "INCPTYREH": "Property income (D4), received by Households", "INCPTYPAH": "Property income (D4), paid by Households",
                "TXDIRPAH": "Current taxes on income and wealth (D5), paid by Households", 
                "SCNETREH": "Net social contribution (D61), received by Households", "SCNETPAH": "Net social contribution (D61), paid by Households",
                "SOCBENOTHREH": "Social benefits other than transfers in kind (D62), received by Households", "SOCBENOTHPAH": "Social benefits other than transfers in kind (D62), paid by Households",
                "SOCBENKIREH": "Social benefits in kind (D63), received by households", "SOCBENKIPAH": "Social benefits in kind (D63), paid by households",
                "CURTRANSREH": "Other current transfers (D7), received by Households", "CURTRANSPAH": "Other current transfers (D7), received by Households",
                "INCGDISPH": "Households' gross disposable income (B6G)"}

for reg in CMPEMPA_r.index.unique():
    for key, value in dict_var_cmph.items():
        scl_nm = f"sh{key.lower()}"
        var_nm = f"{key}_{reg}"
        val[f"{BASEYEAR}"] = CMPEMPA_r.loc[(reg)].sum()*scl_hhinc.loc[(reg,scl_nm)].sum()
        dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": value, "unit": "million current euros", "idt": "", "eqs": ""}


# Calculation of variables proportional to INCGDISP
dict_var_inch = {"ADJPENREH": "Adjustment for the change in pension entitlements (D8), received by Households", "ADJPENPAH": "Adjustment for the change in pension entitlements (D8), paid by Households",
                "GSAVH": "Households' gross saving (B8G)",
                "KTRANSREH": "Capital transfers (D9), received by Households", "KTRANSPAH": "Capital transfers (D9), paid by Households",
                "FIXCAPPAH": "Households' gross capital formation (P5)",
                "DSTOCKVH": "Changes in inventories (P52), for Households",
                "DVALUEVH": "Acquisitions less disposals of valuables (P53) for Households", 
                "ACLDNFNPAH": "Acquisitions less disposals of non-financial non-produced assets (NP), paid by Households",
                "NLENDBH": "Households' net lending (+) / net borrowing (-)"}

for reg in CMPEMPA_r.index.unique():
    for key, value in dict_var_inch.items():
        scl_nm = f"sh{key.lower()}"
        var_nm = f"{key}_{reg}"
        var_inc = f"INCGDISPH_{reg}"
        val[f"{BASEYEAR}"] = dict_var_sa[var_inc]["val"][f"{BASEYEAR}"] *scl_hhinc.loc[(reg,scl_nm)].sum()
        dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": value, "unit": "million current euros", "idt": "", "eqs": ""}

# Remaining variables
op_var = ["Operating surplus: Consumption of fixed capital", "Operating surplus: Rents on land", "Operating surplus: Royalties on resources", "Operating surplus: Remaining net operating surplus"]
GOPSUP = exio19.satellite.F.loc[exio19.satellite.F.index.get_level_values("stressor").isin(op_var)]
GOPSUP_r = GOPSUP.sum().groupby(["region"]).sum()
for reg in GOPSUP_r.index.unique():
    var_nm=  f"GOPSUPH_{reg}"
    val[f"{BASEYEAR}"] = scl_hhinc.loc[(reg,"shgopsuph")].sum()*GOPSUP_r.loc[(reg)].sum()
    dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": "Households' gross operating surplus and mixed income (B2G & B3G)", "unit": "million current euros", "idt": "", "eqs": ""}

DINVV_r = mat_gfcf_dinv.groupby("col_reg").sum()
for reg in DINVV_r.index.unique():
    var_nm=  f"DINVVH_{reg}"
    val[f"{BASEYEAR}"] = scl_hhinc.loc[(reg,"shdinvvh")].sum()*DINVV_r.loc[(reg)].sum()
    dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": "Households' gross fixed capital formation", "unit": "million current euros", "idt": "", "eqs": ""}


## CAUTION
# Re-calculation of DSTOCKVH & NLENDBH to ensure consistency 
for reg in DINVV_r.index.unique():
    val[f"{BASEYEAR}"] = dict_var_sa[f"FIXCAPPAH_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"DINVVH_{reg}"]["val"][f"{BASEYEAR}"] 
    var_nm = f"DSTOCKVH_{reg}"
    dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": "Households' changes in inventories (P52) and acquisitions less disposals of valuables (P53)", "unit": "million current euros", "idt": "", "eqs": ""}
    val[f"{BASEYEAR}"] = dict_var_sa[f"GSAVH_{reg}"]["val"][f"{BASEYEAR}"]+dict_var_sa[f"KTRANSREH_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"KTRANSPAH_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"DINVVH_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"DSTOCKVH_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"ACLDNFNPAH_{reg}"]["val"][f"{BASEYEAR}"]
    var_nm = f"NLENDBH_{reg}"
    dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": "Households' net lending (+) / net borrowing (-))", "unit": "million current euros", "idt": "", "eqs": ""}


#### Calculation of Goverment accounts

In [ ]:
# Loading pre-calculated scalars (from OECD "" and processed)
path_scl_govinc = wk_path + "Data_Raw/Non-Fin_Accounts/"
csv_scl_govinc = os.path.join(path_scl_govinc, "GOV_scl_inc_2019.csv")
scl_govinc = pd.read_csv(csv_scl_govinc, sep=";", dtype={0: str}, header=[0], index_col=[0])

# Calculation of variables proportional to CONSVG
# Sum all CMPEMPA by country
CONSGV_r = exio19.Y.loc[:,(exio19.Y.columns.get_level_values("category") == "CONSGV")].sum()
for reg in CONSGV_r.index.get_level_values("region").unique():
    var_nm = f"CONSGV_{reg}"
    val[f"{BASEYEAR}"] = CONSGV_r.loc[(reg)].sum()
    dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": "Total government' final expenditure", "unit": "million current euros", "idt": "", "eqs": ""}

# Variables and comments
dict_var_consg = {"TXPROIMPREG": "Taxes on production and imports (D2), received by Government", 
                 "SUBTOTPAG": "Subsidies on production and imports (D3), paid by Government",
                 "INCPTYREG": "Property income (D4), received by Government",
                 "INCPTYPAG": "Property income (D4), paid by Government",
                 "SCNETREG": "Net social contribution (D61), received by Government",
                 "SCNETPAG": "Net social contribution (D61), paid by Government",
                 "SOCBENOTHREG": "Social benefits other than transfers in kind (D62), received by Government",
                 "SOCBENOTHPAG": "Social benefits other than transfers in kind (D62), paid by Government",
                 "SOCBENKIREG": "Social benefits in kind (D63), received by Government",
                 "SOCBENKIPAG": "Social benefits in kind (D63), paid by Government",
                 "CURTRANSREG": "Other current transfers (D7), received by Government",
                 "CURTRANSPAG": "Other current transfers (D7), received by Government",
                 "INCGDISPG": "Government's gross disposable income (B6G)"}

for reg in CONSGV_r.index.get_level_values("region").unique():
    for key, value in dict_var_consg.items():
        scl_nm = f"sh{key.lower()}"
        var_nm = f"{key}_{reg}"
        val[f"{BASEYEAR}"] = CONSGV_r.loc[(reg)].sum()*scl_govinc.loc[(reg,scl_nm)].sum()
        dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": value, "unit": "million current euros", "idt": "", "eqs": ""}


# Calculation of variables proportional to INCGDISP
dict_var_incg = {"ADJPENREG": "Adjustment for the change in pension entitlements (D8), received by Government",
                 "ADJPENPAG": "Adjustment for the change in pension entitlements (D8), paid by Government",
                 "GSAVG": "Government's gross saving (B8G)",
                 "KTRANSREG": "Capital transfers (D9), received by Government",
                 "KTRANSPAG": "Capital transfers (D9), paid by Government",
                 "FIXCAPPAG": "Government gross capital formation (P5)",
                 "DSTOCKVG": "Changes in inventories (P52) for Government",
                 "DVALUEVG": "Acquisitions less disposals of valuables (P53) for Government", 
                 "ACLDNFNPAG": "Acquisitions less disposals of non-financial non-produced assets (NP), paid by Government",
                 "NLENDBG": "Government's net lending (+) / net borrowing (-)"}

for reg in CONSGV_r.index.get_level_values("region").unique():
    for key, value in dict_var_incg.items():
        scl_nm = f"sh{key.lower()}"
        var_nm = f"{key}_{reg}"
        var_inc = f"INCGDISPG_{reg}"
        val[f"{BASEYEAR}"] = dict_var_sa[var_inc]["val"][f"{BASEYEAR}"]*scl_govinc.loc[(reg,scl_nm)].sum()
        dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": value, "unit": "million current euros", "idt": "", "eqs": ""}
      

# Remaining variables
for reg in GOPSUP_r.index.unique():
    var_nm =  f"GOPSUPG_{reg}"
    val[f"{BASEYEAR}"] = scl_govinc.loc[(reg,"shgopsupg")].sum()*GOPSUP_r.loc[(reg)].sum()
    dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": "Government's gross operating surplus and mixed income (B2G & B3G)", "unit": "million current euros", "idt": "", "eqs": ""}

for reg in DINVV_r.index.unique():
    var_nm =  f"DINVVG_{reg}"
    val[f"{BASEYEAR}"] = scl_govinc.loc[(reg,"shdinvvg")].sum()*DINVV_r.loc[(reg)].sum()
    dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": "Government's gross fixed capital formation", "unit": "million current euros", "idt": "", "eqs": ""}


## CAUTION
# Re-calculation of DSTOCKVG & NLENDBG to ensure consistency 
for reg in DINVV_r.index.unique():
    val[f"{BASEYEAR}"] = dict_var_sa[f"FIXCAPPAG_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"DINVVG_{reg}"]["val"][f"{BASEYEAR}"]
    var_nm = f"DSTOCKVG_{reg}"
    dict_var[var_nm]  = {"val": val.copy(), "cmt": "Government's changes in inventories (P52) and acquisitions less disposals of valuables (P53)", "unit": "million current euros", "idt": "", "eqs": ""}
    val[f"{BASEYEAR}"] = dict_var_sa[f"GSAVG_{reg}"]["val"][f"{BASEYEAR}"]+dict_var_sa[f"KTRANSREG_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"KTRANSPAG_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"DINVVG_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"DSTOCKVG_{reg}"]["val"][f"{BASEYEAR}"]-dict_var_sa[f"ACLDNFNPAG_{reg}"]["val"][f"{BASEYEAR}"]
    var_nm = f"NLENDBG_{reg}"
    dict_var_sa[var_nm]  = {"val": val.copy(), "cmt": "Government's net lending (+) / net borrowing (-)", "unit": "million current euros", "idt": "", "eqs": ""}



#### Store variables from sectors' accounts and comments into iode objects

In [ ]:
for key in dict_var_sa.keys():
    io.variables[f"{key}"] = np.nan
    io.variables[f"{key}", f"{LOADYEAR}"] = dict_var_sa[f"{key}"]["val"][f"{BASEYEAR}"]
    io.comments[f"{key}"] = dict_var_sa[f"{key}"]["cmt"] + "; Unit:" + dict_var_sa[f"{key}"]["unit"]  


## Scalars calculation

### Scalars for addressed demands calculation

In [30]:
# Create df for dinvv by demanding region and sector for each supplying sector (aggregated by supplying region) 
mat_gfcf_ss = mat_gfcf.groupby("row_sec").sum()
mat_intcons_ss = exio19.Z.groupby("sector").sum()
mat_enr_ss = exio19.Z.loc[(exio19.Z.index.get_level_values('sector').isin(lst_enr_sect))].groupby("sector").sum() 


In [ ]:
# Importation of the calculated Investment tables rs*ss/rd*sd

# Calculation of the share of intermediate consumption (without energy) and energy consumption (total intermediate excluded)
dict_scl_io = dict()

# Create df for dinvv by demanding region and sector for each supplying sector (aggregated by supplying region) 
mat_gfcf_ss = mat_gfcf.groupby("row_sec").sum()
mat_intcons_ss = exio19.Z.groupby("sector").sum()
mat_enr_ss = exio19.Z.loc[(exio19.Z.index.get_level_values('sector').isin(lst_enr_sect))].groupby("sector").sum() 
mat_mat_ss = mat_intcons_ss.sub(mat_enr_ss, fill_value=0)

# Scalars' calculation
for rd in exio19.Z.index.get_level_values("region").unique():
    for sd in exio19.Z.index.get_level_values("sector").unique():
        for ss in exio19.Z.columns.get_level_values("sector").unique():
            # Intermadiate consumption (excl. energy)
            if dict_var[f"DMATV_{rd}_{sd}"]["val"] == 0:
                mmat_rd_sd_ss = 0 
            else:
                mmat_rd_sd_ss = mat_mat_ss.loc[ss,(rd,sd)]/dict_var[f"DMATV_{rd}_{sd}"]["val"][f"{BASEYEAR}"]    
            dict_scl_io[f"mmat_{rd.lower()}_{sd}_{ss}"] = {"val": mmat_rd_sd_ss, "cmt": f"Share of intermediate consumption (except energy) from region {rd} and industry n°{sd} demanded to industry n°{ss}"}
            
            # Gross fixed capital formation
            if dict_var[f"DINVV_{rd}_{sd}"]["val"][f"{BASEYEAR}"] == 0:
                minv_rd_sd_ss = 0
            else:
                minv_rd_sd_ss = mat_gfcf_ss.loc[ss,(rd,sd)]/mat_gfcf_ss.loc[:,(rd,sd)].sum()
            dict_scl_io[f"minv_{rd.lower()}_{sd}_{ss}"] = {"val": minv_rd_sd_ss, "cmt": f"Share of gross fixed capital formation from region {rd} and industry n°{sd} demanded to industry n°{ss}"}    
        
        for ss in lst_enr_sect:
            # Energy consumption
            if dict_var[f"DENRTOTV_{rd}_{sd}"]["val"][f"{BASEYEAR}"] == 0:
                menr_rd_sd_ss = 0 
            else:
                # menr_rd_sd_ss = addci_enr_tot.loc[ss,(rd,sd)]/dict_var[f"DENRTOTV_{rd}_{sd}"]["val"]
                menr_rd_sd_ss = mat_enr_ss.loc[ss,(rd,sd)]/dict_var[f"DENRTOTV_{rd}_{sd}"]["val"][f"{BASEYEAR}"]  
            dict_scl_io[f"menr_{rd.lower()}_{sd}_{ss}"] = {"val": menr_rd_sd_ss, "cmt": f"Share of energy consumption from region {rd} and industry n°{sd} demanded to industry n°{ss}"}
       

#### Store scalars and comments into iode objects

In [ ]:
for key in dict_scl_io.keys():
    io.scalars[f"{key}"] = dict_scl_io[f"{key}"]["val"]
    io.comments[f"{key}"] = dict_scl_io[f"{key}"]["cmt"]


In [ ]:
path_save = wk_path + "/Data/"
io.variables.save(f"{path_save}Data_v0.85.var")
io.identities.save(f"{path_save}Data_v0.85.idt")
io.equations.save(f"{path_save}Data_v0.85.eqs")
io.scalars.save(f"{path_save}Data_v0.85.scl")
io.comments.save(f"{path_save}Data_v0.85.cmt")